# COMP8760 2024-25 Toychain Class

In [ ]:
import hashlib

prev_hash = "your_previous_hash"
name = "your_name"
nonce = 0
i = 1
while True:
  test = f"{prev_hash}{name}{nonce}"
  result = hashlib.sha256(test.encode()).hexdigest()
  if result.startswith("0"*i):
    print(test)
    print(result)
    print(nonce)
  nonce += 1

In [ ]:
import hashlib
import concurrent.futures

# Define your initial input
prev_hash = "previous_hash"
name = "name"
leading_zeros = 1  # Number of leading zeros required
found = False  # Shared flag to stop other threads when a result is found

# Define a worker function to search for the hash with leading zeros
def find_hash(start_nonce, step):
    global found
    nonce = start_nonce
    target_prefix = "0" * leading_zeros

    while not found:
        test = f"{prev_hash}{name}{nonce}"
        result = hashlib.sha256(test.encode()).hexdigest()

        if result.startswith(target_prefix):
            found = True
            print(f"Found hash: {result}")
            print(f"Nonce: {nonce}")
            return result, nonce

        nonce += step  # Each thread jumps by `step` to avoid overlapping nonces

    return None, None

# Run the hashing search in parallel
num_threads = 4  # Adjust based on the number of CPU cores available
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Each thread starts at a different nonce and skips by `num_threads` each time
    futures = [executor.submit(find_hash, i, num_threads) for i in range(num_threads)]

    for future in concurrent.futures.as_completed(futures):
        result, nonce = future.result()
        if result:
            break  # Stop once we find a valid hash